In [ ]:
!pip install -q selenium
!apt install -qq chromium-chromedriver sqlite3
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [23]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
urls = {
  "primary": "https://overframe.gg/tier-list/primary-weapons/",
  "secondary": "https://overframe.gg/tier-list/secondary-weapons/",
  "melee": "https://overframe.gg/tier-list/melee-weapons/"
} 

In [16]:
from selenium.webdriver.common.by import By
import pandas as pd

def getTier(url):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get(url)

  tierContainers = wd.find_elements(by=By.CLASS_NAME, value="TierList_tierContainer__psIt4")
  wStorage = []
  tStorage = []

  for tierContainer in tierContainers:
    currTier = tierContainer.find_element(By.CLASS_NAME, "TierList_tierTitle__PdTC9").text
    weapons = tierContainer.find_elements(By.TAG_NAME, "figcaption")
    for weapon in weapons:
      wStorage.append(weapon.text)
      tStorage.append(currTier)

  d = {
      "weapon": wStorage,
      "tier": tStorage,
  }
  df = pd.DataFrame(data=d)

  return df

### Getting data

In [18]:
priFrame = getTier(urls["primary"])
priFrame

,weapon,tier
0,KUVA ZARR,S Tier - Prime Time
1,NATARUK,S Tier - Prime Time
2,PROBOSCIS CERNOS,S Tier - Prime Time
3,KUVA BRAMMA,S Tier - Prime Time
4,CEDO,S Tier - Prime Time
...,...,...
153,LATRON,D Tier - Weak Picks
154,BRATON,D Tier - Weak Picks
155,HIND,D Tier - Weak Picks
156,DERA,D Tier - Weak Picks


In [20]:
secFrame = getTier(urls["secondary"])
secFrame

,weapon,tier
0,KUVA NUKOR,S Tier - Prime Time
1,LAETUM,S Tier - Prime Time
2,SPORELACER,S Tier - Prime Time
3,TENET CYCRON,S Tier - Prime Time
4,EPITAPH,S Tier - Prime Time
...,...,...
125,BALLISTICA,D Tier - Weak Picks
126,BRONCO,D Tier - Weak Picks
127,TWIN VIPERS,D Tier - Weak Picks
128,MK1-FURIS,D Tier - Weak Picks


In [22]:
melFrame = getTier(urls["melee"])
melFrame

,weapon,tier
0,NIKANA PRIME,S Tier - Prime Time
1,KRONEN PRIME,S Tier - Prime Time
2,GUANDAO PRIME,S Tier - Prime Time
3,STROPHA,S Tier - Prime Time
4,GLAIVE PRIME,S Tier - Prime Time
...,...,...
188,MK1-FURAX,D Tier - Weak Picks
189,KESTREL,D Tier - Weak Picks
190,FURAX,D Tier - Weak Picks
191,KOGAKE,D Tier - Weak Picks


### SQL Part

In [30]:
import sqlite3
 
conn = sqlite3.connect('test.db')

In [31]:
priFrame.to_sql('primary_weapons', con=conn)
secFrame.to_sql('secondary_weapons', con=conn)
melFrame.to_sql('melee_weapons', con=conn)

In [36]:
!sqlite3 test.db .dump >> database.sql
!tail -n 10 database.sql

INSERT INTO melee_weapons VALUES(187,'FANG','D Tier - Weak Picks');
INSERT INTO melee_weapons VALUES(188,'MK1-FURAX','D Tier - Weak Picks');
INSERT INTO melee_weapons VALUES(189,'KESTREL','D Tier - Weak Picks');
INSERT INTO melee_weapons VALUES(190,'FURAX','D Tier - Weak Picks');
INSERT INTO melee_weapons VALUES(191,'KOGAKE','D Tier - Weak Picks');
INSERT INTO melee_weapons VALUES(192,'ANKYROS','D Tier - Weak Picks');
CREATE INDEX "ix_primary_weapons_index"ON "primary_weapons" ("index");
CREATE INDEX "ix_secondary_weapons_index"ON "secondary_weapons" ("index");
CREATE INDEX "ix_melee_weapons_index"ON "melee_weapons" ("index");
COMMIT;
